In [1]:
# Libraries

# Data handling 
import pandas as pd
import numpy as np

# Feature transformation
from sklearn import preprocessing 

# Regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
from scipy import stats

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
# read the file
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df = pd.read_csv('./files/training.zip', sep = ',', compression = 'zip')

In [3]:
# remove outliers
# Outlier removal 
df = df[df['claim_amount'] <= 9000]

In [5]:
# calculate y
df.head()

,id_policy,year,pol_no_claims_discount,pol_coverage,pol_duration,pol_sit_duration,pol_pay_freq,pol_payd,pol_usage,drv_sex1,drv_age1,drv_age_lic1,drv_drv2,drv_sex2,drv_age2,drv_age_lic2,vh_make_model,vh_age,vh_fuel,vh_type,vh_speed,vh_value,vh_weight,population,town_surface_area,claim_amount
0,PL000000,1.0,0.332,Med2,5,1,Monthly,No,WorkPrivate,M,35.0,16.0,Yes,F,26.0,1.0,aparvvfowrjncdhp,8.0,Gasoline,Tourism,174.0,11040.0,1143.0,1270.0,33.1,0.0
1,PL042495,1.0,0.000,Med2,6,1,Monthly,No,WorkPrivate,M,60.0,41.0,No,0,NaN,NaN,aparvvfowrjncdhp,10.0,Diesel,Tourism,174.0,11040.0,1143.0,1290.0,51.3,0.0
2,PL042496,1.0,0.196,Med1,2,1,Yearly,Yes,Retired,M,55.0,35.0,Yes,F,57.0,38.0,iwhqpdfuhrsxyqxe,8.0,Diesel,Commercial,150.0,14159.0,1193.0,1020.0,262.8,0.0
3,PL042497,1.0,0.000,Med2,8,5,Yearly,No,WorkPrivate,F,54.0,31.0,No,0,NaN,NaN,kvcddisqpkysmvvo,4.0,Gasoline,Tourism,149.0,17233.0,1012.0,180.0,219.7,0.0
4,PL042498,1.0,0.000,Med1,2,2,Yearly,No,Retired,F,65.0,38.0,No,0,NaN,NaN,tdgkjlphosocwbgu,13.0,Gasoline,Tourism,200.0,19422.0,1315.0,30.0,70.3,0.0


In [34]:
# New variable binary 0 means not claim and 1 means claim 
df['claim'] = df['claim_amount'].apply(lambda x: 0 if x == 0.0 else 1)

# Counter of the df
df['counts'] = 1

In [36]:
df_policyid = df.groupby('id_policy')[['claim_amount','claim','counts']].sum()

In [42]:
df_policyid['occurence']  =  df_policyid['claim'] / df_policyid['counts']
df_policyid['severity']  =  df_policyid['claim_amount'] / df_policyid['counts']

In [ ]:
vehvars = []
drivvars = ['drv_sex1','drv_age1','drv_age_lic1',]
polvars = ['pol_no_claims_discount','pol_coverage','pol_duration','pol_sit_duration','pol_payd','pol_usage']

In [4]:
df['claim_amount'].head(100)

0        0.00
1        0.00
2        0.00
3        0.00
4        0.00
5        0.00
6        0.00
7        0.00
8        0.00
9        0.00
10       0.00
11       0.00
12       0.00
13       0.00
14     339.84
15    1236.00
16       0.00
17       0.00
18       0.00
19       0.00
20     209.98
21       0.00
22       0.00
23       0.00
24       0.00
25       0.00
26       0.00
27       0.00
28       0.00
29       0.00
30       0.00
31     224.33
32       0.00
33       0.00
34       0.00
35       0.00
36       0.00
37       0.00
38       0.00
39       0.00
40       0.00
41       0.00
42       0.00
43       0.00
44       0.00
45    1343.41
46       0.00
47       0.00
48     130.23
49       0.00
50       0.00
51       0.00
52       0.00
53       0.00
54     390.03
55       0.00
56       0.00
57       0.00
58       0.00
59       0.00
60       0.00
61       0.00
62       0.00
63     699.54
64       0.00
65       0.00
66       0.00
67       0.00
68       0.00
69       0.00
70       0.00
71    

In [ ]:
# data cleaning
# New variable binary 0 means not claim and 1 means claim 
df['claim'] = df['claim_amount'].apply(lambda x: 0 if x == 0.0 else 1)

# Counter of the df
df['counts'] = 1

# Variable to study if there is a correlation between pol duration and the situation
df['conditions_minus'] = df['pol_duration'] - df['pol_sit_duration']

# 
df['conditions_prop'] = df['pol_sit_duration'] / df['pol_duration']

# Variable to study if it is possible to create a variable using the duration of the polize and the situation of this
df['pol_duration+sit_duration'] = df['pol_sit_duration'] * df['conditions_prop']

# Function to define the age of the second driver
def drv2(x):
    if np.isnan(x['drv_age2']):
        return x['drv_age1']
    else:
        return x['drv_age2']

# Function to define the years of licence of the second driver
def drv2_time(x):
    if np.isnan(x['drv_age_lic2']):
        return x['drv_age_lic1']
    else:
        return x['drv_age_lic2']

# Functions applied
df['drv_age2_2'] = df.apply(lambda x: drv2(x), axis = 1)
df['drv_age_lic2_2'] = df.apply(lambda x: drv2_time(x), axis = 1)
df['age_dif'] = df['drv_age1'] - df['drv_age2_2'] 
df['lic_dif'] = df['drv_age_lic1'] - df['drv_age_lic2_2'] 
df['lic_dif'] = df['drv_age_lic1'] - df['drv_age_lic2_2'] 

In [ ]:
# log linear variables

In [ ]:
# generate a weight factor 

In [ ]:
# offset variables